In [124]:
# !pip install pygame
import pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))

In [134]:
player1_controls = {
    "UP": pygame.K_w,
    "DOWN": pygame.K_s,
    "LEFT": pygame.K_a,
    "RIGHT": pygame.K_d,
    "ACTION": pygame.K_e
}

player2_controls = {
    "UP": pygame.K_UP,
    "DOWN": pygame.K_DOWN,
    "LEFT": pygame.K_LEFT,
    "RIGHT": pygame.K_RIGHT,
    "ACTION": pygame.K_SPACE
}


from Player import Player, Players
from Board import player1_start, player2_start
from constants import PLAYER1_GRAPHIC, PLAYER2_GRAPHIC
import os
os.chdir('..')

Player1 = Player(player1_start, PLAYER1_GRAPHIC, player1_controls)
Players.add(Player1)

Player2 = Player(player2_start, PLAYER2_GRAPHIC, player2_controls)
Players.add(Player2)

In [135]:
from Objects import Walls, Floors, CBoards, Fryers
from Foods import Resources

def draw():
    screen.fill((255, 255, 255)) 
    Floors.draw(screen)
    Walls.draw(screen)
    Resources.draw(screen)
    Players.draw(screen)

In [136]:
import numpy as np

def get_state():
    pixel_array = pygame.surfarray.array3d(screen)
    # Transpose to (height, width, channels)
    pixel_array = np.transpose(pixel_array, (1, 0, 2))
    
    #Normalize so that pixel values are from 0 to 1, not 0 to 255 as in RGB
    pixel_array = pixel_array / 255.0
    pixel_array = np.expand_dims(pixel_array, axis=0)

In [137]:
from tensorflow.keras.models import load_model
import os
os.chdir('/Users/michal.maslowski/Documents/GitHub/OvercookedAI-coop')

Misha = load_model('Misha.h5') #Load the weights of the model

In [138]:
from Foods import Menu

In [139]:
def get_state():
    '''
    Returns the list of:
    - visual_data - normalized pixel values of the screen
    - numerical_data, containing in respective order:
        - the state of the Menu (see: Menu.get_state())
        - the state of Player 1's and Player 2's hands (see: Player.get_state())
    
    '''
    
    visual_data = pygame.surfarray.array3d(screen)
    visual_data = np.transpose(visual_data, (1, 0, 2)) #Change from width, height, color channel to height, width, color channel
    visual_data = visual_data / 255. #Normalize pixels from 0 to 1 for easier training
    visual_data = np.expand_dims(visual_data, 0) #Add the batch_size dimension

    numerical_data = np.concatenate((Menu.get_state(), Player1.get_state(), Player2.get_state()))
    numerical_data = np.expand_dims(numerical_data, 0)
    
    return [visual_data, numerical_data]




In [140]:
time_per_game = int(input("Please enter the length of the game, in seconds (default: 180)"))
start_time = pygame.time.get_ticks()
from constants import START_X, END_X, START_Y, END_Y

    
def update(actions):
    elapsed_time = (pygame.time.get_ticks() - start_time) / 1000
    if(elapsed_time < time_per_game):
        Players.update(actions) 
        CBoards.update()
        Fryers.update()


    else:
        #Game finished; display a large "GAME OVER" sign over a frozen frame
        font = pygame.font.SysFont("comicsansms", 100)
        game_over_surface = font.render("GAME OVER", True, (255, 0, 0))
        game_over_rect = game_over_surface.get_rect(center=((START_X+END_X)//2, (START_Y + END_Y)//2))
        screen.blit(game_over_surface, game_over_rect)

In [141]:
def get_actions(misha_playing):
    if not misha_playing:
        return pygame.key.get_pressed()
    
    else:
        #Create a blank slate dictionary (as if all the keys were unpressed)
        keys = [key for key in dir(pygame) if key.startswith('K_')]
        keys_dict = {getattr(pygame, key): False for key in keys}
        #Now, set all relevant keys as pressed
        
        actions = []
        q_values = Misha.predict(get_state())
        q_values = q_values[0] #Get rid of the batch_size dimension: goes from (1, 10) to (10,)
        p1_values = q_values[0:5]
        p1_action = list(player1_controls)[p1_values.argmax()] #Get the action from player 1 that maximizes the q value
        keys_dict[p1_action] = True
        
        if (p1_action == player1_controls["ACTION"]): #If the best value for player 1 involves using an object:
            p1_action_no2 = list(player1_controls)[p1_values[:4].argmax()] #Get the 2nd best action
            keys_dict[p1_action_no2] = True
            
        #ditto as above for player 2
        p2_values = q_values[5:10]
        p2_action = list(player2_controls)[p2_values.argmax()] #Get the action from player 1 that maximizes the q value
        keys_dict[p2_action] = True
        if (p2_action == player2_controls["ACTION"]): #If the best value for player 1 involves using an object:
            p2_action_no2 = list(player2_controls)[p1_values[:4].argmax()] #Get the 2nd best action
            keys_dict[p2_action_no2] = True
        
        return keys_dict

In [143]:
misha_playing = True #Bot vs human play
running = True
clock = pygame.time.Clock()
tick = 0
game_score = 0


while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break

    if(not running):
        break
    
    draw()
    
    actions = get_actions(misha_playing)
    update(actions)
    pygame.display.flip()
    tick += 1
    clock.tick(60)
    if(tick % 60 == 0):
        pass
        
    
        
pygame.display.quit()
pygame.quit()
exit()

1/1 [==============================] - 0s 20ms/step


UnboundLocalError: local variable 'action_made' referenced before assignment